## [1] kaggle 사용을 위한 setting

In [0]:
!pip uninstall -y kaggle
!pip install --upgrade pip
!pip install kaggle==1.5.6

Uninstalling kaggle-1.5.6:
  Successfully uninstalled kaggle-1.5.6
     |████████████████████████████████| 1.5MB 3.4MB/s 
  Found existing installation: pip 19.3.1
    Uninstalling pip-19.3.1:
      Successfully uninstalled pip-19.3.1
     |████████████████████████████████| 58 kB 2.1 MB/s 
  Created wheel for kaggle: filename=kaggle-1.5.6-py3-none-any.whl size=72859 sha256=c7e6c4d4cce8f143983369edd57f2026b2ba6eb118d0f1e9ec24e1c10b1feccd
  Stored in directory: /root/.cache/pip/wheels/01/3e/ff/77407ebac3ef71a79b9166a8382aecf88415a0bcbe3c095a01
Successfully built kaggle


In [0]:
!mkdir -p ~/.kaggle                   #캐글 폴더 생성
!cp kaggle.json ~/.kaggle/            # 폴더에 kaggle.json을 복붙하기
! chmod 600 ~/.kaggle/kaggle.json     # 복붙한 것의 권한을 주기
!kaggle -v

Kaggle API 1.5.6


In [0]:
!kaggle competitions download -c regression-cabbage-price

!unzip regression-cabbage-price.zip   #폴더이름.zip

  0% 0.00/29.7k [00:00<?, ?B/s]
100% 29.7k/29.7k [00:00<00:00, 26.1MB/s]
Archive:  regression-cabbage-price.zip
  inflating: submit_sample.csv       
  inflating: test_cabbage_price.csv  
  inflating: train_cabbage_price.csv  


## [2] 데이터 로더

In [0]:
import numpy as np
import torch
import torch.optim as optim


xy_train = np.loadtxt('train_cabbage_price.csv', delimiter=",", dtype=np.float32,skiprows=1, usecols=range(1,6))

# skiprows=   : 사용하지 않을 컬럼 
# usecol = range(0~6) : 사용할 컬럼 5개. 1~5      0보다 크거나 같고 6보다 작다

x_train=torch.from_numpy(xy_train[:,0:-1])
y_train=torch.from_numpy(xy_train[:,[-1]])


In [0]:
print(x_train)
print(y_train)       #가격

#print(x_train.shape)
#print(y_train.shape)

tensor([[ -4.9000, -11.0000,   0.9000,   0.0000],
        [ -3.1000,  -5.5000,   5.5000,   0.8000],
        [ -2.9000,  -6.9000,   1.4000,   0.0000],
        ...,
        [  2.9000,  -2.1000,   8.0000,   0.0000],
        [  2.9000,  -1.6000,   7.1000,   0.6000],
        [  2.1000,  -2.0000,   5.8000,   0.4000]])
tensor([[2123.],
        [2123.],
        [2123.],
        ...,
        [2901.],
        [2901.],
        [2901.]])


## [3] 모델 학습

In [0]:
W = torch.zeros((4, 1), requires_grad=True)
b = torch.zeros(1, requires_grad=True)

# optimizer 설정
optimizer = optim.SGD([W, b], lr=1e-5)

nb_epochs = 10000
for epoch in range(nb_epochs + 1):
    
    # H(x) 계산
    # 원래 위의 계산 :  hypothesis = x1_train * w1 + x2_train * w2 + x3_train * w3 + b
    # Matrix 연산을 이용한 계산
    hypothesis = x_train.matmul(W) + b # or .mm or @    

    # cost 계산
    cost = torch.mean((hypothesis - y_train) ** 2)

    # cost로 H(x) 개선 (W,b 갱신)
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 1000 == 0:
         print('Epoch {:4d}/{} hypothesis: {} Cost: {:.6f}'.format(
            epoch, nb_epochs, hypothesis.squeeze().detach(), cost.item()
         ))


Epoch    0/10000 hypothesis: tensor([0., 0., 0.,  ..., 0., 0., 0.]) Cost: 12610017.000000
Epoch 1000/10000 hypothesis: tensor([ 565.7002,  973.6077,  482.8234,  ..., 1337.3057, 1195.5322,
        1006.8253]) Cost: 3220925.750000
Epoch 2000/10000 hypothesis: tensor([1319.5573, 1585.1024,  993.6948,  ..., 1782.9357, 1577.5721,
        1364.2250]) Cost: 2648012.250000
Epoch 3000/10000 hypothesis: tensor([1790.6487, 1968.4285, 1313.1221,  ..., 2058.8489, 1815.1150,
        1586.5706]) Cost: 2423400.000000
Epoch 4000/10000 hypothesis: tensor([2085.4590, 2209.5017, 1513.3942,  ..., 2230.9795, 1963.6235,
        1725.6838]) Cost: 2334645.000000
Epoch 5000/10000 hypothesis: tensor([2270.0762, 2361.6482, 1639.2308,  ..., 2338.7471, 2056.7424,
        1813.0127]) Cost: 2299132.750000
Epoch 6000/10000 hypothesis: tensor([2385.7263, 2458.1333, 1718.4919,  ..., 2406.3596, 2115.2598,
        1867.9917]) Cost: 2284506.750000
Epoch 7000/10000 hypothesis: tensor([2458.1941, 2519.7644, 1768.5939,  ..., 

## [4] 정답예측. 데이터에 대한 TEST


In [0]:
# 평균 온도:-2.7 최저 온도:-6.6 최고 온도:2.0 강수량:0.1  이때의 배추값은?

x_test= torch.tensor([-2.7, -6.6, 2.0, 0.1])
y_test=x_test.matmul(W)+b
print(y_test)

tensor([1926.6575], grad_fn=<AddBackward0>)


캐글의 TEST 셋을 가지고 예측값을 구하기

In [0]:
# test는 y값이 없음.    usecols=range(1,5)
xy_test=np.loadtxt('test_cabbage_price.csv',delimiter=",", dtype=np.float32,skiprows=1, usecols=range(1,5))

test_x_data=torch.from_numpy(xy_test) #xy_test를 numpy에서 tensor로
print(test_x_data)  #6,4

tensor([[13.5000,  8.0000, 19.2000,  2.6000],
        [ 1.1000, -3.0000,  9.5000,  0.8000],
        [12.0000,  5.1000, 19.6000,  0.0000],
        [16.6000, 11.9000, 22.5000,  0.1000],
        [22.4000, 17.8000, 27.4000,  0.5000],
        [18.8000, 13.9000, 24.4000,  0.0000]])


In [0]:
# 위에서 x_train(training데이터)로 구했던 W,b로 만든 모델에
# test data (캐글에서 주어진)를 이용해서 값을 예측하기 prediction 

prediction= test_x_data.matmul(W)+b

print(test_x_data)
print(prediction)

tensor([[13.5000,  8.0000, 19.2000,  2.6000],
        [ 1.1000, -3.0000,  9.5000,  0.8000],
        [12.0000,  5.1000, 19.6000,  0.0000],
        [16.6000, 11.9000, 22.5000,  0.1000],
        [22.4000, 17.8000, 27.4000,  0.5000],
        [18.8000, 13.9000, 24.4000,  0.0000]])
tensor([[3343.0706],
        [3093.7808],
        [3932.7683],
        [3268.5181],
        [3302.7729],
        [3330.6177]], grad_fn=<AddBackward0>)


## [5] 캐글에 제출하기


- test 데이터 셋의 포맷에 맞추기.
submit_sample.csv


In [0]:
import pandas as pd

submit=pd.read_csv('submit_sample.csv')
submit               # id와 expected의 헤더를 가지고 있다.

,Id,Expected
0,0,1000
1,1,1000
2,2,1000
3,3,1000
4,4,1000
5,5,1000


In [0]:
submit['Expected'][0]

1000

In [0]:
for i in range(len(prediction)):
  submit['Expected'][i]=prediction[i].item()

submit

,Id,Expected
0,0,3343
1,1,3093
2,2,3932
3,3,3268
4,4,3302
5,5,3330


In [0]:
submit.to_csv('submit.csv',mode='w', index=False)

#w: write모드 기존에 있던 csv에 덮어씌우기.

In [0]:
! kaggle competitions submit -c regression-cabbage-price -f submit.csv -m 'third submit 16010064 이주희'

100% 54.0/54.0 [00:01<00:00, 33.4B/s]
Successfully submitted to Regression : Cabbage price

In [0]:
# kaggle의 leader보드에서 확인하기